In [1]:
import numpy as np
import pandas as pd

import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

from config import fetch_data

## **RETENTION ACTIONS**

### Retention Action Priority Matrix

In [2]:
retention_action_priority_matrix_query = " SELECT * FROM gold.analytics_customer_retention_actions ORDER BY expected_roi DESC"
retention_action_priority_matrix_df = fetch_data(retention_action_priority_matrix_query)

median_success = retention_action_priority_matrix_df['estimated_success_rate_pct'].median()
median_clv = retention_action_priority_matrix_df['clv_at_risk'].median()

# Create scatter plot
fig = px.scatter(
    retention_action_priority_matrix_df,
    x='estimated_success_rate_pct',
    y='clv_at_risk',
    size='churn_risk_pct',
    color='action_priority',
    hover_name='recommended_action',
    hover_data={
        'clv_at_risk': ':$,.2f',
        'estimated_success_rate_pct': ':.0f%',
        'churn_risk_pct': ':.1f%',
        'expected_roi': ':$,.2f',
        'recommended_channel': True,
        'recommended_timeline': True
    },
    title='Retention Action Priority Matrix',
    labels={
        'estimated_success_rate_pct': 'Estimated Success Rate (%)',
        'clv_at_risk': 'CLV at Risk ($)',
        'churn_risk_pct': 'Churn Risk (%)',
        'action_priority': 'Action Priority'
    },
    color_discrete_map={1: '#EF553B', 2: '#FFA15A', 3: '#00CC96'},
    category_orders={'action_priority': [1, 2, 3]},
    template='plotly_white'
)

# Add quadrant lines
fig.add_hline(y=median_clv, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=median_success, line_dash="dash", line_color="gray", opacity=0.5)

# Add quadrant labels
fig.add_annotation(x=median_success*0.5, y=median_clv*1.5, text="High CLV, Low Success", showarrow=False, font=dict(size=12))
fig.add_annotation(x=median_success*1.5, y=median_clv*1.5, text="High CLV, High Success", showarrow=False, font=dict(size=12))
fig.add_annotation(x=median_success*0.5, y=median_clv*0.5, text="Low CLV, Low Success", showarrow=False, font=dict(size=12))
fig.add_annotation(x=median_success*1.5, y=median_clv*0.5, text="Low CLV, High Success", showarrow=False, font=dict(size=12))

# Improve layout
fig.update_layout(
    height=650,
    width=850,
    legend_title_text='Action Priority',
    title=dict(x=0.5),
    xaxis=dict(tickformat='.0%'),
    yaxis=dict(tickprefix='$', separatethousands=True)
)

fig.show()

In [3]:
retention_action_priority_matrix_df

,customer_key,customer_natural_key,customer_segment,churn_risk_pct,clv_at_risk,recommended_action,action_priority,recommended_channel,estimated_success_rate_pct,days_since_login,transaction_count_90d,active_accounts,past_due_accounts,unresolved_issues,negative_interactions,customer_lifetime_value,generated_at,expected_roi,recommended_timeline
0,82debd8a12b498e765a11a8e51159440,5357,Business,91.0,99916.0,URGENT: Resolve Service Issues,1,Phone Call,78.0,1140.0,0,2,0,1,1,99916,2025-12-27 07:58:02.783243+00:00,38817.24,Within 48 hours
1,496e05e1aea0a9c4655800e8a7b9ea28,305,Premium,64.0,99871.0,URGENT: Resolve Service Issues,1,Phone Call,78.0,2077.0,0,2,0,1,0,99871,2025-12-27 07:58:02.783243+00:00,38799.69,Within 48 hours
2,6af97d7deea1a1d2c76c5c512e66700b,7032,Business,91.0,99820.0,URGENT: Resolve Service Issues,1,Phone Call,78.0,4654.0,0,0,0,1,0,99820,2025-12-27 07:58:02.783243+00:00,38779.80,Within 48 hours
3,0771fc6f0f4b1d7d1bb73bbbe14e0e31,1632,Business,95.0,99817.0,URGENT: Resolve Service Issues,1,Phone Call,78.0,3439.0,0,2,0,1,0,99817,2025-12-27 07:58:02.783243+00:00,38778.63,Within 48 hours
4,177fdb0c18f85f1fa5dcfc76d9b2cdd9,13622,Business,71.0,99708.0,URGENT: Resolve Service Issues,1,Phone Call,78.0,NaN,0,4,0,1,0,99708,2025-12-27 07:58:02.783243+00:00,38736.12,Within 48 hours
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8147,a97ea3db450da9d3c22cc7b158f4d515,13216,Business,99.0,1023.0,Re-engagement Campaign,2,Direct Mail,42.0,2177.0,0,4,0,0,0,1023,2025-12-27 07:58:02.783243+00:00,164.83,Within 1 week
8148,65b9d1b6673cfd8602f9f9765265bc78,20167,Premium,75.0,1182.0,URGENT: Resolve Service Issues,1,Phone Call,52.0,4330.0,0,1,0,1,0,1182,2025-12-27 07:58:02.783243+00:00,157.32,Within 48 hours
8149,e2065cb56f5533494522c46a72f1dfb0,3500,Affluent,78.0,1219.0,Win-Back Offer,2,Direct Mail,42.0,NaN,0,2,0,0,1,1219,2025-12-27 07:58:02.783243+00:00,155.99,Within 1 week
8150,05c0ae7c6b605732401aa102a62970b3,19405,Affluent,63.0,1274.0,Re-engagement Campaign,3,Direct Mail,28.0,1286.0,0,1,0,0,0,1274,2025-12-27 07:58:02.783243+00:00,128.36,Within 2 weeks


### Retention Action Breakdown

In [4]:
retention_action_breakdown_query = """
    SELECT recommended_action, action_priority, customer_segment,
        COUNT(*) as customer_count,
        SUM(clv_at_risk) as total_clv_at_risk,
        AVG(expected_roi) as avg_roi
    FROM gold.analytics_customer_retention_actions
    GROUP BY recommended_action, action_priority, customer_segment
"""

retention_action_breakdown_df = fetch_data(retention_action_breakdown_query)

fig = px.sunburst(
    retention_action_breakdown_df,
    path=['action_priority', 'recommended_action', 'customer_segment'],
    values='customer_count',
    color='avg_roi',
    hover_data={
        'customer_count': ':,',
        'total_clv_at_risk': ':$,.2f',
        'avg_roi': ':$,.2f'
    },
    title='Retention Action Strategy Breakdown',
    color_continuous_scale='RdYlGn',
    labels={'customer_count': 'Customers', 'avg_roi': 'Avg ROI'}
)

fig.update_layout(height=700)

In [5]:
retention_action_breakdown_df

,recommended_action,action_priority,customer_segment,customer_count,total_clv_at_risk,avg_roi
0,Win-Back Offer,3,Premium,7,40668.0,1026.820000
1,Re-engagement Campaign,3,Premium,51,253726.0,999.485294
2,Re-engagement Campaign,2,Premium,565,26627922.0,9773.760726
3,Cross-Sell Campaign,3,Mass Market,1,5455.0,1397.850000
4,URGENT: Resolve Service Issues,1,Premium,430,21298058.0,18465.383419
5,Loyalty Program Enrollment,3,Affluent,1,1875.0,410.000000
6,Re-engagement Campaign,3,Affluent,55,352265.0,1249.160909
7,Loyalty Program Enrollment,1,Affluent,7,408176.0,13954.605714
8,Re-engagement Campaign,2,Affluent,609,29858799.0,10132.280920
9,Re-engagement Campaign,1,Premium,765,42284999.0,11386.101843


### Retention ROI Waterfall

In [6]:
retention_roi_waterfall_query = """
    SELECT recommended_action, 
        SUM(expected_roi) as total_expected_roi,
        COUNT(*) as customer_count
    FROM gold.analytics_customer_retention_actions
    WHERE action_priority <= 2
    GROUP BY recommended_action
    ORDER BY total_expected_roi DESC
"""

retention_roi_waterfall_df = fetch_data(retention_roi_waterfall_query)

total_roi = retention_roi_waterfall_df['total_expected_roi'].sum()

# Create waterfall chart
fig = go.Figure(go.Waterfall(
    name="ROI",
    orientation="v",
    measure=["relative"] * len(retention_roi_waterfall_df) + ["total"],
    x=retention_roi_waterfall_df['recommended_action'].tolist() + ["Total Expected ROI"],
    y=retention_roi_waterfall_df['total_expected_roi'].tolist() + [0],
    text=[f"${v:,.0f}" for v in retention_roi_waterfall_df['total_expected_roi']] + [f"${total_roi:,.0f}"],
    textposition="outside",
    connector={"line": {"color": "gray", "dash": "dot"}},
    increasing={"marker": {"color": "#00CC96"}},
    totals={"marker": {"color": "#636EFA"}},
    hovertemplate='<b>%{x}</b><br>ROI: $%{y:,.2f}<br>Contribution: %{customdata:.1f}%<extra></extra>',
    customdata=[(v / total_roi * 100) for v in retention_roi_waterfall_df['total_expected_roi']] + [100]
))

# Update layout
fig.update_layout(
    title=dict(
        text="Expected ROI from Retention Actions (Priority 1 & 2)",
        x=0.5,
        xanchor='center'
    ),
    yaxis_title="Expected ROI ($)",
    height=550,
    template="plotly_white",
    margin=dict(l=60, r=40, t=80, b=60)
)

fig.show()

In [7]:
retention_roi_waterfall_df

,recommended_action,total_expected_roi,customer_count
0,Re-engagement Campaign,58558269.65,5432
1,URGENT: Resolve Service Issues,33243295.11,1777
2,Win-Back Offer,6456592.89,612
3,Loyalty Program Enrollment,970893.00,73
4,Cross-Sell Campaign,134829.83,10


## **PRODUCT RECOMMENDATIONS**

### Product Recommendation Funnel

In [8]:
product_recommendation_funnel_query = """
    SELECT recommended_product, 
        SUM(expected_value) as total_value, 
        COUNT(*) as customer_count,
        AVG(propensity_to_accept_pct) as avg_propensity
    FROM gold.analytics_product_recommendation
    GROUP BY recommended_product
    ORDER BY total_value DESC
"""
product_recommendation_funnel_df = fetch_data(product_recommendation_funnel_query)

# Create funnel chart
fig = go.Figure(go.Funnel(
    y=product_recommendation_funnel_df['recommended_product'],
    x=product_recommendation_funnel_df['total_value'],
    textinfo="value+percent initial",
    marker={"color": px.colors.sequential.Teal[::-1]},  # reverse for descending visual
    hovertemplate=(
        '<b>%{y}</b><br>'
        'Expected Value: $%{x:,.2f}<br>'
        'Customers: %{customdata[0]:,}<br>'
        'Avg Propensity: %{customdata[1]:.0f}%<extra></extra>'
    ),
    customdata=product_recommendation_funnel_df[['customer_count', 'avg_propensity']].values
))

# Update layout for presentation
fig.update_layout(
    title=dict(
        text="Product Recommendation Expected Value Funnel",
        x=0.5,
        xanchor='center'
    ),
    height=600,
    template='plotly_white',
    margin=dict(l=100, r=50, t=80, b=50),
    yaxis=dict(title='Recommended Product'),
    xaxis=dict(title='Total Expected Value ($)', tickprefix='$', separatethousands=True)
)

fig.show()

### Product Recommendation Matrix

In [9]:
product_recommendation_matrix_query = "SELECT * FROM gold.analytics_product_recommendation ORDER BY expected_value DESC"
product_recommendation_matrix_df = fetch_data(product_recommendation_matrix_query)

# Compute median lines for quadrants
median_score = product_recommendation_matrix_df['recommendation_score'].median()
median_value = product_recommendation_matrix_df['expected_value'].median()

# Create scatter plot
fig = px.scatter(
    product_recommendation_matrix_df,
    x='recommendation_score',
    y='expected_value',
    size='propensity_to_accept_pct',
    color='campaign_priority',
    hover_data={
        'recommended_product': True,
        'customer_segment': True,
        'current_products': True,
        'expected_annual_revenue': ':$,.2f',
        'propensity_to_accept_pct': ':.0f%'
    },
    title='Product Recommendation Matrix: Score vs Expected Value (Top 500)',
    labels={
        'recommendation_score': 'Recommendation Score',
        'expected_value': 'Expected Value ($)',
        'propensity_to_accept_pct': 'Propensity to Accept (%)'
    },
    color_discrete_map={
        'High Priority': '#EF553B',
        'Medium Priority': '#FFA15A',
        'Low Priority': '#00CC96'
    },
    template='plotly_white'
)

# Add quadrant lines
fig.add_hline(y=median_value, line_dash="dash", line_color="gray", opacity=0.5)
fig.add_vline(x=median_score, line_dash="dash", line_color="gray", opacity=0.5)

# Add quadrant labels
fig.add_annotation(x=median_score*0.5, y=median_value*1.5, text="High Value, Low Score", showarrow=False, font=dict(size=12))
fig.add_annotation(x=median_score*1.5, y=median_value*1.5, text="High Value, High Score", showarrow=False, font=dict(size=12))
fig.add_annotation(x=median_score*0.5, y=median_value*0.5, text="Low Value, Low Score", showarrow=False, font=dict(size=12))
fig.add_annotation(x=median_score*1.5, y=median_value*0.5, text="Low Value, High Score", showarrow=False, font=dict(size=12))

# Improve layout
fig.update_layout(
    height=650,
    width=850,
    title=dict(x=0.5),
    legend_title_text='Campaign Priority',
    xaxis=dict(tickformat='.2f'),
    yaxis=dict(tickprefix='$', separatethousands=True)
)

fig.show()

In [10]:
product_recommendation_matrix_df

,customer_key,customer_natural_key,customer_segment,current_products,product_count,recommended_product,recommendation_score,expected_value,propensity_to_accept_pct,expected_annual_revenue,recommended_campaign_channel,campaign_priority,total_balance,generated_at
0,82a008560ab8fa4bd01994e2ff52bb6a,17600,Premium,LOAN,2,Personal Loan,94.0,440.0,55,800,Personal Banker Call,High Priority,-670018.84,2025-12-27 07:58:06.118665+00:00
1,e515df0d202ae52fcebb14295743063b,1030,Premium,"CREDIT, DEPOSIT, LOAN",4,Personal Loan,94.0,440.0,55,800,Personal Banker Call,High Priority,-226414.24,2025-12-27 07:58:06.118665+00:00
2,626fbe83f8c7f2670b90a773b6dbf17c,7552,Premium,"DEPOSIT, LOAN",2,Personal Loan,94.0,440.0,55,800,Personal Banker Call,High Priority,-266735.76,2025-12-27 07:58:06.118665+00:00
3,afd53be629a8800e6447030f2e0961f7,16823,Premium,"DEPOSIT, LOAN",2,Personal Loan,94.0,440.0,55,800,Personal Banker Call,High Priority,-153466.58,2025-12-27 07:58:06.118665+00:00
4,afd208b1d98bc70e0aedc93bb4371c14,13155,Premium,"CREDIT, DEPOSIT",3,Personal Loan,94.0,440.0,55,800,Personal Banker Call,High Priority,-2176.85,2025-12-27 07:58:06.118665+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13849,81e68999106d6798eca552cbb9337751,16912,Mass Market,"CREDIT, DEPOSIT, LOAN",3,Credit Card,90.0,60.0,25,240,Personal Banker Call,High Priority,-227219.36,2025-12-27 07:58:06.118665+00:00
13850,d0aae9539e4dd0bd618e5d2598f18707,12774,Mass Market,"CREDIT, DEPOSIT, LOAN",4,Credit Card,90.0,60.0,25,240,Personal Banker Call,High Priority,-231292.64,2025-12-27 07:58:06.118665+00:00
13851,b151ce4935a3c2807e1dd9963eda16d8,4653,Mass Market,"CREDIT, LOAN",3,Credit Card,90.0,60.0,25,240,Personal Banker Call,High Priority,-732033.76,2025-12-27 07:58:06.118665+00:00
13852,7392ea4ca76ad2fb4c9c3b6a5c6e31e3,9327,Business,"DEPOSIT, INVESTMENT",3,Credit Card,81.0,60.0,25,240,Personal Banker Call,High Priority,578537.47,2025-12-27 07:58:06.118665+00:00


## **CREDIT LIMIT OPTIMIZATION**

### Credit Limit Optimization Scatter

In [11]:
credit_limit_optimization_scatter_query = "SELECT * FROM gold.analytics_credit_limit_optimization WHERE recommended_action != 'No Change'"
credit_limit_optimization_scatter_df = fetch_data(credit_limit_optimization_scatter_query)


fig = px.scatter(
    credit_limit_optimization_scatter_df,
    x='current_limit',
    y='recommended_limit',
    color='recommended_action',
    size=abs(credit_limit_optimization_scatter_df['limit_change']),
    hover_data={
        'current_limit': ':$,.2f',
        'recommended_limit': ':$,.2f',
        'limit_change_pct': ':.1f%',
        'current_utilization_pct': ':.1f%',
        'projected_utilization_pct': ':.1f%',
        'adjustment_rationale': True
    },
    title='Credit Limit Optimization Recommendations',
    labels={
        'current_limit': 'Current Limit ($)',
        'recommended_limit': 'Recommended Limit ($)'
    },
    color_discrete_map={
        'Increase Limit': '#00CC96',
        'Decrease Limit': '#EF553B',
        'Review Required': '#FFA15A'
    }
)

# Add diagonal line (no change)
max_val = max(credit_limit_optimization_scatter_df['current_limit'].max(), credit_limit_optimization_scatter_df['recommended_limit'].max())
fig.add_trace(go.Scatter(
    x=[0, max_val],
    y=[0, max_val],
    mode='lines',
    line=dict(dash='dash', color='gray', width=2),
    name='No Change Line',
    hoverinfo='skip'
))

fig.update_layout(height=600)
fig.show()

In [12]:
credit_limit_optimization_scatter_df

,account_key,account_natural_key,customer_key,customer_segment,credit_score,current_limit,recommended_limit,limit_change,limit_change_pct,recommended_action,adjustment_rationale,risk_level,current_utilization_pct,projected_utilization_pct,account_age_months,late_payments_12m,avg_monthly_spend,generated_at


### Credit Limit Impact Analysis

In [13]:
credit_limit_impact_analysis_query = """
    SELECT recommended_action,
        COUNT(*) as account_count,
        AVG(limit_change) as avg_change,
        AVG(limit_change_pct) as avg_change_pct,
        AVG(current_utilization_pct) as avg_current_util,
        AVG(projected_utilization_pct) as avg_projected_util
    FROM gold.analytics_credit_limit_optimization
    WHERE recommended_action != 'No Change'
    GROUP BY recommended_action
"""
credit_limit_impact_analysis_df = fetch_data(credit_limit_impact_analysis_query)

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Account Count by Action",
                    "Avg Utilization: Current vs Projected")
)

# Account count
fig.add_trace(
    go.Bar(
        x=credit_limit_impact_analysis_df['recommended_action'],
        y=credit_limit_impact_analysis_df['account_count'],
        marker_color=['#00CC96', '#EF553B', '#FFA15A'],
        text=credit_limit_impact_analysis_df['account_count'],
        texttemplate='%{text:,}',
        textposition='outside',
        showlegend=False,
        hovertemplate='<b>%{x}</b><br>Accounts: %{y:,}<extra></extra>'
    ),
    row=1, col=1
)

# Utilization comparison
fig.add_trace(
    go.Bar(
        name='Current',
        x=credit_limit_impact_analysis_df['recommended_action'],
        y=credit_limit_impact_analysis_df['avg_current_util'],
        marker_color='lightblue',
        hovertemplate='<b>%{x}</b><br>Current: %{y:.1f}%<extra></extra>'
    ),
    row=1, col=2
)

fig.add_trace(
    go.Bar(
        name='Projected',
        x=credit_limit_impact_analysis_df['recommended_action'],
        y=credit_limit_impact_analysis_df['avg_projected_util'],
        marker_color='darkblue',
        hovertemplate='<b>%{x}</b><br>Projected: %{y:.1f}%<extra></extra>'
    ),
    row=1, col=2
)

fig.update_xaxes(title_text="Action", row=1, col=1)
fig.update_xaxes(title_text="Action", row=1, col=2)
fig.update_yaxes(title_text="Account Count", row=1, col=1)
fig.update_yaxes(title_text="Utilization (%)", row=1, col=2)

fig.update_layout(
    title_text="Credit Limit Optimization Impact Analysis",
    height=500,
    barmode='group'
)
fig.show()

In [14]:
credit_limit_impact_analysis_df

,recommended_action,account_count,avg_change,avg_change_pct,avg_current_util,avg_projected_util


## **MARKETING BUDGET ALLOCATION**

### Marketing Budget Allocation sankey

In [15]:
marketing_budget_allocation_sankey_query = "SELECT * FROM gold.analytics_marketing_budget_allocation ORDER BY recommended_budget DESC"
marketing_budget_allocation_sankey_df = fetch_data(marketing_budget_allocation_sankey_query)

# Prepare Sankey nodes and links
sources = []
targets = []
values = []

labels = ['Total Budget'] + (
    marketing_budget_allocation_sankey_df['campaign_type'] + " - " +
    marketing_budget_allocation_sankey_df['channel_group']
).tolist()

for i, row in marketing_budget_allocation_sankey_df.iterrows():
    sources.append(0)  # From Total Budget
    targets.append(i + 1)
    values.append(row['recommended_budget'])

# Color nodes by efficiency
median_efficiency = marketing_budget_allocation_sankey_df['efficiency_score'].median()
colors = ['lightblue'] + [
    'green' if e > median_efficiency else 'orange' 
    for e in marketing_budget_allocation_sankey_df['efficiency_score']
]

# Create Sankey diagram
fig = go.Figure(go.Sankey(
    node=dict(
        pad=15,
        thickness=20,
        label=labels,
        color=colors
    ),
    link=dict(
        source=sources,
        target=targets,
        value=values,
        hovertemplate='%{source.label} → %{target.label}<br>Budget: $%{value:,.0f}<extra></extra>'
    )
))

# Layout improvements
fig.update_layout(
    title=dict(
        text="Optimal Marketing Budget Allocation (Top 20 Campaigns)",
        x=0.5,
        xanchor='center'
    ),
    height=700,
    template='plotly_white'
)

fig.show()

In [16]:
marketing_budget_allocation_sankey_df

,campaign_type,channel_group,target_segment,campaign_count,total_budget,total_conversions,avg_roi,avg_cpa,avg_conversion_rate,efficiency_score,recommended_budget,budget_change,expected_revenue,expected_conversions,priority_rank,budget_recommendation,generated_at
0,Email,Digital Direct,Affluent,1,411986.45,3608.0,2.960000,55.800000,10.690000,0.567068,303123.70,-108862.75,897246.15,5432.0,1,Increase Budget,2025-12-27 07:58:05.234180+00:00
1,TV,Traditional Media,Business,3,828426.09,8463.0,1.190000,219.313333,100.376667,0.544646,291138.32,14996.29,346454.60,1327.0,2,Increase Budget,2025-12-27 07:58:05.234180+00:00
2,Online Display,Other,Business,1,213800.85,1579.0,0.760000,336.810000,112.620000,0.254123,135840.36,-77960.49,103238.67,403.0,3,Increase Budget,2025-12-27 07:58:05.234180+00:00
3,Social Media,Digital Advertising,Affluent,1,167153.74,1368.0,2.150000,487.830000,40.710000,0.179420,95908.20,-71245.54,206202.63,197.0,4,Increase Budget,2025-12-27 07:58:05.234180+00:00
4,TV,Traditional Media,Affluent,1,318882.16,4934.0,2.800000,368.750000,18.010000,0.136754,73101.18,-245780.98,204683.30,198.0,5,Maintain Budget,2025-12-27 07:58:05.234180+00:00
5,Radio,Traditional Media,Affluent,3,643685.38,7039.0,1.693333,350.870000,11.496667,0.055484,29658.75,-184903.04,50222.15,85.0,6,Maintain Budget,2025-12-27 07:58:05.234180+00:00
6,Radio,Traditional Media,Premium,2,427936.03,4973.0,1.370000,441.095000,15.995000,0.049679,26555.68,-187412.34,36381.28,60.0,7,Maintain Budget,2025-12-27 07:58:05.234180+00:00
7,Email,Digital Direct,Premium,2,699971.77,5379.0,1.175000,436.805000,12.285000,0.033046,17664.86,-332321.03,20756.21,40.0,8,Maintain Budget,2025-12-27 07:58:05.234180+00:00
8,Radio,Traditional Media,Business,1,190975.14,1843.0,0.440000,101.010000,6.630000,0.028880,15437.84,-175537.30,6792.65,153.0,9,Maintain Budget,2025-12-27 07:58:05.234180+00:00
9,Online Display,Other,Premium,2,679693.00,4023.0,0.860000,401.415000,9.410000,0.020160,10776.54,-329069.96,9267.82,27.0,10,Decrease Budget,2025-12-27 07:58:05.234180+00:00


### Marketing Efficienct Scatter

In [17]:
marketing_efficiency_scatter_query = "SELECT * FROM gold.analytics_marketing_budget_allocation"
marketing_efficiency_scatter_df = fetch_data(marketing_efficiency_scatter_query)

# Ensure bubble sizes are positive
marketing_efficiency_scatter_df['expected_conversions'] = marketing_efficiency_scatter_df['expected_conversions'].clip(lower=1)

# Optional: scale bubble sizes for better visibility
size_scale = 1.5  # adjust for presentation

# Create scatter plot
fig = px.scatter(
    marketing_efficiency_scatter_df,
    x='efficiency_score',
    y='recommended_budget',
    size='expected_conversions',
    color='budget_recommendation',
    hover_data={
        'campaign_type': True,
        'channel_group': True,
        'avg_roi': ':.2f',
        'avg_conversion_rate': ':.2f%',
        'expected_revenue': ':$,.2f',
        'expected_conversions': ':,'
    },
    title='Marketing Campaign Efficiency vs Recommended Budget',
    labels={
        'efficiency_score': 'Efficiency Score',
        'recommended_budget': 'Recommended Budget ($)',
        'expected_conversions': 'Expected Conversions'
    },
    color_discrete_map={
        'Increase Budget': '#00CC96',  # green
        'Decrease Budget': '#EF553B',  # red
        'Maintain Budget': '#636EFA'   # blue
    },
    size_max=50,  # maximum bubble size
    template='plotly_white',
    opacity=0.8
)

# Enhance layout
fig.update_layout(
    height=650,
    width=900,
    title=dict(x=0.5, font=dict(size=20)),
    xaxis=dict(
        title='Efficiency Score',
        tickformat='.2f',
        showgrid=True,
        gridcolor='lightgray',
        zeroline=True,
        zerolinecolor='gray'
    ),
    yaxis=dict(
        title='Recommended Budget ($)',
        tickprefix='$',
        separatethousands=True,
        showgrid=True,
        gridcolor='lightgray',
        zeroline=True,
        zerolinecolor='gray'
    ),
    legend_title_text='Budget Recommendation'
)

# Add outlines to bubbles for contrast
fig.update_traces(marker=dict(line=dict(width=1, color='DarkSlateGrey')))

fig.show()

In [18]:
marketing_efficiency_scatter_df

,campaign_type,channel_group,target_segment,campaign_count,total_budget,total_conversions,avg_roi,avg_cpa,avg_conversion_rate,efficiency_score,recommended_budget,budget_change,expected_revenue,expected_conversions,priority_rank,budget_recommendation,generated_at
0,Email,Digital Direct,Affluent,1,411986.45,3608.0,2.960000,55.800000,10.690000,0.567068,303123.70,-108862.75,897246.15,5432.0,1,Increase Budget,2025-12-27 07:58:05.234180+00:00
1,TV,Traditional Media,Business,3,828426.09,8463.0,1.190000,219.313333,100.376667,0.544646,291138.32,14996.29,346454.60,1327.0,2,Increase Budget,2025-12-27 07:58:05.234180+00:00
2,Online Display,Other,Business,1,213800.85,1579.0,0.760000,336.810000,112.620000,0.254123,135840.36,-77960.49,103238.67,403.0,3,Increase Budget,2025-12-27 07:58:05.234180+00:00
3,Social Media,Digital Advertising,Affluent,1,167153.74,1368.0,2.150000,487.830000,40.710000,0.179420,95908.20,-71245.54,206202.63,197.0,4,Increase Budget,2025-12-27 07:58:05.234180+00:00
4,TV,Traditional Media,Affluent,1,318882.16,4934.0,2.800000,368.750000,18.010000,0.136754,73101.18,-245780.98,204683.30,198.0,5,Maintain Budget,2025-12-27 07:58:05.234180+00:00
5,Radio,Traditional Media,Affluent,3,643685.38,7039.0,1.693333,350.870000,11.496667,0.055484,29658.75,-184903.04,50222.15,85.0,6,Maintain Budget,2025-12-27 07:58:05.234180+00:00
6,Radio,Traditional Media,Premium,2,427936.03,4973.0,1.370000,441.095000,15.995000,0.049679,26555.68,-187412.34,36381.28,60.0,7,Maintain Budget,2025-12-27 07:58:05.234180+00:00
7,Email,Digital Direct,Premium,2,699971.77,5379.0,1.175000,436.805000,12.285000,0.033046,17664.86,-332321.03,20756.21,40.0,8,Maintain Budget,2025-12-27 07:58:05.234180+00:00
8,Radio,Traditional Media,Business,1,190975.14,1843.0,0.440000,101.010000,6.630000,0.028880,15437.84,-175537.30,6792.65,153.0,9,Maintain Budget,2025-12-27 07:58:05.234180+00:00
9,Online Display,Other,Premium,2,679693.00,4023.0,0.860000,401.415000,9.410000,0.020160,10776.54,-329069.96,9267.82,27.0,10,Decrease Budget,2025-12-27 07:58:05.234180+00:00


## **OPERATIONAL OPTIMIZATION**

In [19]:
branch_staffing_optimization_query = "SELECT * FROM gold.analytics_branch_staffing_optimization ORDER BY ABS(staff_change) DESC LIMIT 20"
branch_staffing_optimization_df = fetch_data(branch_staffing_optimization_query)
fig = go.Figure()

# Current staff
fig.add_trace(go.Bar(
    name='Current Staff',
    x=branch_staffing_optimization_df['branch_name'],
    y=branch_staffing_optimization_df['current_staff_estimate'],
    marker_color='lightgray',
    hovertemplate='<b>%{x}</b><br>Current: %{y}<extra></extra>'
))

# Recommended staff
fig.add_trace(go.Bar(
    name='Recommended Staff',
    x=branch_staffing_optimization_df['branch_name'],
    y=branch_staffing_optimization_df['recommended_staff'],
    marker_color=['#00CC96' if c >
                    0 else '#EF553B' for c in branch_staffing_optimization_df['staff_change']],
    text=branch_staffing_optimization_df['staff_change'],
    texttemplate='%{text:+.0f}',
    textposition='outside',
    hovertemplate='<b>%{x}</b><br>Recommended: %{y}<br>Change: %{text}<extra></extra>'
))

fig.update_layout(
    title="Branch Staffing Optimization (Top 20 by Change Magnitude)",
    xaxis_title="Branch",
    yaxis_title="Staff Count",
    barmode='group',
    height=600
)

fig.show()

In [20]:
branch_staffing_optimization_df

,branch_id,branch_name,region,total_transactions,branch_transactions,service_interactions,current_staff_estimate,recommended_staff,generated_at,staff_change,annual_cost_impact,current_transactions_per_staff,optimal_transactions_per_staff,staffing_recommendation
